In [1]:
import os
import ast
import folium

from freeman.aiddd.data_manager import read_data

In [2]:
BASE_KEY_FILE_PATH = os.path.join(
    os.path.expanduser('~'), 'projects', 'data'
)
VWORLD_KEY_FILE_PATH = os.path.join(BASE_KEY_FILE_PATH, 'vworld_key.txt')
with open(VWORLD_KEY_FILE_PATH, 'r') as vkf:
    vk_lines = vkf.readlines()
VWORLD_KEY = vk_lines[0].replace('\n', '')

VWORLD_TILE_TYPE='png'
VWORLD_ATTR = 'Vworld'

In [3]:
dtype_options = {'pole12_comp_no': str, 'pole12_xy': object}
df_data = read_data('bd-all-merge', dtype=dtype_options)

In [4]:
select_columns = ['cons_no', 'line_cnts', 'pole_cnts', 'distance_sum']
pole_columns = [
    col for col in df_data.columns if col.startswith('pole') and col[4].isdigit()
]
select_columns += pole_columns

In [5]:
df_path = df_data[select_columns].copy()

In [6]:
MAX_POLE_COUNTS = 12
dict_path = {}

for _, row in df_path.iterrows():
    cons_no = row.cons_no
    xy_path = []
    distance_path = []
    for idx in range(1, MAX_POLE_COUNTS):
        column_name_xy = f'pole{idx}_xy'
        column_name_distance = f'pole{idx+1}_distance'
        if row[column_name_xy] == '[0, 0]' or row[column_name_xy] == '0.0':
            continue
        xy_path.append(ast.literal_eval(row[column_name_xy]))
        distance_path.append(row[column_name_distance])
    column_name_xy = f'pole{MAX_POLE_COUNTS}_xy'
    if row[column_name_xy] != '0.0':
        xy_path.append(ast.literal_eval(row[column_name_xy]))
    distance_path.append(row['distance_sum'])
        
    dict_path[cons_no] = {
        'xy_path': xy_path,
        'distance_path': distance_path
    }

In [7]:
dict_path['477220233833']['xy_path']

[[127.851636237149, 36.0929128045988],
 [127.851287509278, 36.0928070317587],
 [127.850832769222, 36.0926150819813],
 [127.850422150777, 36.0923916542081],
 [127.850008931154, 36.0921141180283],
 [127.84950215476, 36.0919684568562],
 [127.849133333968, 36.0918655327092]]

In [8]:
dict_path['477220233833']['distance_path']

[37.0, 51.0, 49.0, 53.0, 52.0, 36.0, 464]

In [18]:
LABEL_POSITION = 0.0001
def show_pole_and_line(cons_no, VWORLD_LAYER='Base'):
    path = dict_path[cons_no]['xy_path']
    x_values = [point[1] for point in path]
    y_values = [point[0] for point in path]
    min_x, max_x = min(x_values), max(x_values)
    min_y, max_y = min(y_values), max(y_values)
    
    VWORLD_TILES_URL = \
    f'http://api.vworld.kr/req/wmts/1.0.0/' \
    f'{VWORLD_KEY}/{VWORLD_LAYER}/{{z}}/{{y}}/{{x}}.{VWORLD_TILE_TYPE}'
    
    base_map = folium.Map(
        location = [(min_x + max_x)/2, (min_y + max_y)/2], zoom_start=16,
        tiles=VWORLD_TILES_URL, attr=VWORLD_ATTR, 
    )
    
    poly_path = [[y, x] for x, y in path]
    folium.PolyLine(locations=poly_path, color='green').add_to(base_map)
    for idx, point in enumerate(path):
        folium.CircleMarker(location=[point[1],point[0]], radius=3, color='blue').add_to(base_map)
        folium.Marker(
            location=[point[1]+LABEL_POSITION,point[0]+LABEL_POSITION],
            icon=folium.DivIcon(
                html=f"<div style='font-size: 14px; color: red;'>{idx+1}</div>"
            )
        ).add_to(base_map)
    return base_map

In [19]:
show_pole_and_line('474620215283')

In [20]:
show_pole_and_line('70HC20214254')

In [21]:
show_pole_and_line('474620224992')